<a href="https://colab.research.google.com/github/Kajuro/SuwuS/blob/main/Voice_Cloning_RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Colab переделан [Astymic](https://t.me/astymicm)**

**Автор моделей: [Astymic](https://t.me/astymic)**

Оригинальный колаб Kalomaze

##1. Установка + Подключете свой аккаунт Google - нажмите кнопку воспроизведения (необходимо выполнить только один раз за сеанс Colab)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/liujing04/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb)
If you want to open the ORIGINAL Colab go here!

In [1]:
#@title GPU Check
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
#@title Install Dependencies (and load your cached install if it exists to boost times)
# Required Libraries
import os
import csv
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime

#@markdown This will forcefully update dependencies even after the initial install seemed to have functioned.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown This will force temporary storage to be used, so it will download dependencies every time instead of on Drive. Not needed, unless you really want that 160mb storage. (Turned on by default for non-training colab to boost the initial launch speed)
ForceTemporaryStorage = True #@param{type:"boolean"}

# Mounting Google Drive
if not ForceTemporaryStorage:
    from google.colab import drive

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    else:
        print('Drive is already mounted. Proceeding...')

# Function to install dependencies with progress
def install_packages():
    packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
    pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'gradio==3.34.0',
                    'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5',
                    'numba==0.56.4', 'librosa==0.9.2', 'mega.py', 'gdown', 'onnxruntime', 'pyngrok==4.1.12']

    print("Updating and installing system packages...")
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

    print("Updating and installing pip packages...")
    subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

    print('Packages up to date.')

# Function to scan a directory and writes filenames and timestamps
def scan_and_write(base_path, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        for dirpath, dirs, files in os.walk(base_path):
            for filename in files:
                fname = os.path.join(dirpath, filename)
                try:
                    mtime = os.path.getmtime(fname)
                    writer.writerow([fname, mtime])
                except Exception as e:
                    print(f'Skipping irrelevant nonexistent file {fname}: {str(e)}')
    print(f'Finished recording filesystem timestamps to {output_file}.')

# Function to compare files
def compare_files(old_file, new_file):
    old_files = {}
    new_files = {}

    with open(old_file, 'r') as f:
        reader = csv.reader(f)
        old_files = {rows[0]:rows[1] for rows in reader}

    with open(new_file, 'r') as f:
        reader = csv.reader(f)
        new_files = {rows[0]:rows[1] for rows in reader}

    removed_files = old_files.keys() - new_files.keys()
    added_files = new_files.keys() - old_files.keys()
    unchanged_files = old_files.keys() & new_files.keys()

    changed_files = {f for f in unchanged_files if old_files[f] != new_files[f]}

    for file in removed_files:
        print(f'File has been removed: {file}')

    for file in changed_files:
        print(f'File has been updated: {file}')

    return list(added_files) + list(changed_files)

# Check if CachedRVC.tar.gz exists
if ForceTemporaryStorage:
    file_path = '/content/CachedRVC.tar.gz'
else:
    file_path = '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz'

content_file_path = '/content/CachedRVC.tar.gz'
extract_path = '/'

if not os.path.exists(file_path):
    folder_path = os.path.dirname(file_path)
    os.makedirs(folder_path, exist_ok=True)
    print('No cached dependency install found. Attempting to download GitHub backup..')

    try:
        download_url = "https://github.com/kalomaze/QuickMangioFixes/releases/download/release3/CachedRVC.tar.gz"
        !wget -O $file_path $download_url
        print('Download completed successfully!')
    except Exception as e:
        print('Download failed:', str(e))

        # Delete the failed download file
        if os.path.exists(file_path):
            os.remove(file_path)
        print('Failed download file deleted. Continuing manual backup..')

if Path(file_path).exists():
    if ForceTemporaryStorage:
        print('Finished downloading CachedRVC.tar.gz.')
    else:
        print('CachedRVC.tar.gz found on Google Drive. Proceeding to copy and extract...')

    # Check if ForceTemporaryStorage is True and skip copying if it is
    if ForceTemporaryStorage:
         pass
    else:
        shutil.copy(file_path, content_file_path)

    print('Beginning backup copy operation...')

    with tarfile.open(content_file_path, 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join(extract_path, member.name)
            try:
                tar.extract(member, extract_path)
            except Exception as e:
                print('Failed to extract a file (this isn\'t normal)... forcing an update to compensate')
                ForceUpdateDependencies = True
        print(f'Extraction of {content_file_path} to {extract_path} completed.')

    if ForceUpdateDependencies:
        install_packages()
        ForceUpdateDependencies = False
else:
    print('CachedRVC.tar.gz not found. Proceeding to create an index of all current files...')
    scan_and_write('/usr/', '/content/usr_files.csv')

    install_packages()

    scan_and_write('/usr/', '/content/usr_files_new.csv')
    changed_files = compare_files('/content/usr_files.csv', '/content/usr_files_new.csv')

    with tarfile.open('/content/CachedRVC.tar.gz', 'w:gz') as new_tar:
        for file in changed_files:
            new_tar.add(file)
            print(f'Added to tar: {file}')

    os.makedirs('/content/drive/MyDrive/RVC_Cached', exist_ok=True)
    shutil.copy('/content/CachedRVC.tar.gz', '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz')
    print('Updated CachedRVC.tar.gz copied to Google Drive.')
    print('Dependencies fully up to date; future runs should be faster.')

Finished downloading CachedRVC.tar.gz.
Beginning backup copy operation...


EOFError: ignored

In [ ]:
#@title Clone Github Repository
import os

# Change the current directory to /content/
os.chdir('/content/')

# Changes defaults of the infer-web.py
def edit_file(file_path):
    temp_file_path = "/tmp/temp_file.py"
    changes_made = False
    with open(file_path, "r") as file, open(temp_file_path, "w") as temp_file:
        previous_line = ""
        for line in file:
            new_line = line.replace("value=160", "value=128")
            if new_line != line:
                print("Replaced 'value=160' with 'value=128'")
                changes_made = True
            line = new_line

            new_line = line.replace("crepe hop length: 160", "crepe hop length: 128")
            if new_line != line:
                print("Replaced 'crepe hop length: 160' with 'crepe hop length: 128'")
                changes_made = True
            line = new_line

            new_line = line.replace("value=0.88", "value=0.75")
            if new_line != line:
                print("Replaced 'value=0.88' with 'value=0.75'")
                changes_made = True
            line = new_line

            if "label=i18n(\"输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络\")" in previous_line and "value=1," in line:
                new_line = line.replace("value=1,", "value=0.25,")
                if new_line != line:
                    print("Replaced 'value=1,' with 'value=0.25,' based on the condition")
                    changes_made = True
                line = new_line

            if 'choices=["pm", "harvest", "dio", "crepe", "crepe-tiny", "mangio-crepe", "mangio-crepe-tiny"], # Fork Feature. Add Crepe-Tiny' in previous_line:
                if 'value="pm",' in line:
                    new_line = line.replace('value="pm",', 'value="mangio-crepe",')
                    if new_line != line:
                        print("Replaced 'value=\"pm\",' with 'value=\"mangio-crepe\",' based on the condition")
                        changes_made = True
                    line = new_line

            temp_file.write(line)
            previous_line = line

    # After finished, we replace the original file with the temp one
    import shutil
    shutil.move(temp_file_path, file_path)

    if changes_made:
        print("Changes made and file saved successfully.")
    else:
        print("No changes were needed.")

repo_path = '/content/Retrieval-based-Voice-Conversion-WebUI'
if not os.path.exists(repo_path):
    # Clone the latest code from the Mangio621/Mangio-RVC-Fork repository
    !git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
    os.chdir('/content/Mangio-RVC-Fork')
    !git checkout a5b45691957f7a4b2615bcc02ea453490023fac3
    os.chdir('/content/')
    !mv /content/Mangio-RVC-Fork /content/Retrieval-based-Voice-Conversion-WebUI
    edit_file("/content/Retrieval-based-Voice-Conversion-WebUI/infer-web.py")
else:
    print(f"The repository already exists at {repo_path}. Skipping cloning.")

# Download the credentials file for RVC archive sheet
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/stats/
!wget -q https://cdn.discordapp.com/attachments/945486970883285045/1114717554481569802/peppy-generator-388800-07722f17a188.json -O /content/Retrieval-based-Voice-Conversion-WebUI/stats/peppy-generator-388800-07722f17a188.json

# Forcefully delete any existing torchcrepe dependency from an earlier run
!rm -rf /Retrieval-based-Voice-Conversion-WebUI/torchcrepe

# Download the torchcrepe folder from the maxrmorrison/torchcrepe repository
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Retrieval-based-Voice-Conversion-WebUI/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder

# Change the current directory to /content/Retrieval-based-Voice-Conversion-WebUI
os.chdir('/content/Retrieval-based-Voice-Conversion-WebUI')
!mkdir -p pretrained uvr5_weights

In [ ]:
#@title Download the Base Model
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

In [ ]:
#@markdown This will also create an RVC and dataset folders in your drive if they don't already exist.
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)